## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-09-13-00-00 +0000,missionlocal,Cut off,https://missionlocal.org/2025/10/sf-photos-cut...
1,2025-10-09-13-00-00 +0000,wsj,The books have closed on the U.S. government’s...,https://www.wsj.com/economy/tariffs-are-way-up...
2,2025-10-09-13-00-00 +0000,wsj,Previously unreported documents shed new light...,https://www.wsj.com/politics/policy/texas-ken-...
3,2025-10-09-12-58-29 +0000,nypost,Grinning woman hunted for cutting man’s testic...,https://nypost.com/2025/10/09/us-news/grinning...
4,2025-10-09-12-56-25 +0000,nyt,Here is the latest.,https://www.nytimes.com/live/2025/10/09/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
148,trump,53
60,gaza,28
57,deal,27
14,new,20
62,fire,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
186,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...,199
35,2025-10-09-11-51-45 +0000,nypost,Gazans join Israelis in chanting Trump’s name ...,https://nypost.com/2025/10/09/us-news/gazans-j...,153
292,2025-10-08-18-05-37 +0000,nypost,"Trump mulls weekend trip to Egypt, Gaza to sea...",https://nypost.com/2025/10/08/world-news/israe...,138
209,2025-10-08-22-30-24 +0000,wapo,"Trump, Hamas officials say Gaza ceasefire deal...",https://www.washingtonpost.com/world/2025/10/0...,138
141,2025-10-09-04-23-07 +0000,nyt,President Trump Says Israel and Hamas Agree to...,https://www.nytimes.com/2025/10/08/world/middl...,137


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
186,199,2025-10-08-23-17-50 +0000,nypost,Trump announces Israel and Hamas signed off on...,https://nypost.com/2025/10/08/world-news/trump...
61,73,2025-10-09-10-39-00 +0000,wsj,China introduced new controls on rare-earth ex...,https://www.wsj.com/economy/trade/china-impose...
190,73,2025-10-08-23-12-49 +0000,latimes,"After Palisades fire failures, L.A. promises t...",https://www.latimes.com/california/story/2025-...
201,61,2025-10-08-22-51-00 +0000,wsj,President Trump’s declaration that Illinois Go...,https://www.wsj.com/politics/policy/trump-prit...
267,50,2025-10-08-19-36-45 +0000,latimes,Trump sends California National Guard to Chica...,https://www.latimes.com/politics/story/2025-10...
371,49,2025-10-08-13-17-07 +0000,nypost,Manchester synagogue terrorist called cops mid...,https://nypost.com/2025/10/08/world-news/synag...
226,47,2025-10-08-21-33-46 +0000,nypost,Andrew Cuomo would scrap plan to close Rikers ...,https://nypost.com/2025/10/08/us-news/andrew-c...
316,40,2025-10-08-16-42-00 +0000,wsj,Funding for federal agencies lapsed beginning ...,https://www.wsj.com/politics/policy/federal-go...
278,40,2025-10-08-19-07-00 +0000,wsj,Federal Reserve officials were divided over ho...,https://www.wsj.com/economy/central-banking/fe...
181,36,2025-10-08-23-25-18 +0000,nyt,"Texts from Jay Jones, Democratic Attorney Gene...",https://www.nytimes.com/2025/10/08/us/jay-jone...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
